In [25]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [9]:
# Carrega o modelo leve
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [23]:
def gerar_resposta(pergunta, documentos, max_tokens=256):
    # Combina os documentos relevantes como contexto
    contexto = " ".join(documentos)    
    # Prompt para o modelo
    prompt = (
    f"Abaixo está uma descrição de um ou mais acidentes aéreos. "
    f"Com base nessas informações, responda claramente à pergunta.\n\n"
    f"Documentos:\n{contexto}\n\n"
    f"Pergunta: {pergunta}\n"
    f"Resposta:"
)
    
    # Tokeniza o prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Gera a resposta
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    
    # Decodifica
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta

In [11]:
import nbimporter
from retrieval import use_corpus, use_bm25, recuperar_documentos
import pandas as pd

df = pd.read_csv("../data/processed/clean_data.csv")
corpus = use_corpus(df)
bm25 = use_bm25(corpus)


In [24]:
pergunta = "Whats the place with most accidents?"
docs_relevantes = recuperar_documentos(pergunta, bm25, corpus, k=1)
docs_relevantes = [" ".join(doc) if isinstance(doc, list) else doc for doc in docs_relevantes]

resposta = gerar_resposta(pergunta, docs_relevantes)

print("Pergunta:", pergunta)
print("Resposta:", resposta)

Pergunta: Whats the place with most accidents?
Resposta: clarksville , tennessee , united states
